# Trabalho Prático 2

Trabalho realizado pelo grupo 11:

* Beatriz Fernandes Oliveira, PG50942
    
* Bruno Filipe Machado Jardim, PG49997

## Problema 1

1. Construir uma classe Python que implemente um KEM - ElGamal. A classe deve:
    - Inicializar cada instância recebendo  o parâmetro de segurança (tamanho em bits da ordem do grupo cíclico) e gere as chaves pública e privada.
    - Conter funções para encapsulamento e revelação da chave gerada.
    - Construir,  a partir deste KEM e usando a transformação de Fujisaki-Okamoto, um PKE que seja IND-CCA seguro.


### A. 
É necessário garantir que o DLP seja complexo, para isto não basta que o $p$ seja grande, é necessário que o maior fator primo de $(p-1)$ é tambem grande:

Para garantir estas condições o primo $p$ tem de ser gerado de uma determinda forma:
- É gerado um primo q, com pelo menos 160 bits de tamanho
- Geram-se sucessivamente inteiros $p = q*2+1$ e $q$ até que p seja um primo suficientemente grande para satisfazer o parametro de segurança


Optamos por este método pois quando utilizamos o método descrito no [Capítulo 3a: Introdução à Álgebra Abstrata (continuação)](https://paper.dropbox.com/doc/Capitulo-3a-Introducao-a-Algebra-Abstrata-continuacao--B1NHOHi40IhSVblbXMayxPO7Ag-76H4R2e1YWSbWZ210Z8fQ), isto causou com que o parametro $p$ se tornasse extremamente grande e causasse problemas na implementação da transformação Fujisaki-Okamoto.

### B.
As funções de encapsulamento e revelação foram definidas da seguinte forma
- $KEM(\beta) \equiv
\vartheta r \gets \mathbb Z_q \setminus 0 \centerdot
\vartheta \mathsf{key} \gets \beta^r \bmod p \centerdot
\vartheta \mathsf{enc} \gets g^r \bmod p \centerdot
(\mathsf{key},\mathsf{enc}) $

- $KRev(a,\mathsf{enc}) \equiv \mathsf{enc}^a \bmod p$

In [1]:
class ElGamal:
    def __init__(self, size):
        def genPrime():
            q_size = 160
            q = random_prime(ZZ.random_element(2^(q_size-1),2^q_size-1))
            pi = 2*q + 1
            
            while not is_prime(pi) and len(pi.binary()) < self.size:
                q = next_prime(q)
                pi = 2*q + 1
                
            return pi,q
    
        def genParams():
            p,q = genPrime()
            R = GF(p)
            g = R.multiplicative_generator()
            return (p, q, g)
        
        
        
        self.size = size
        self.p, self.q, self.g = genParams()
        
    
    def keyGen(self):
        a = ZZ.random_element(2, (self.q)-1)
        beta = power_mod(self.g, a, self.p)
        return a, beta
    
    def KEM(self, beta, r=None):
        if r is None:
            r = ZZ.random_element(1, (self.q)-1)
        
        key = power_mod(beta, r, self.p)
        enc = power_mod(self.g, r, self.p)
        return key, enc
    
    def KRev(self,a ,enc):
        return power_mod(enc, a, self.p)
        
    

In [2]:
# Generation of the keys
eg = ElGamal(1024)
alice_pvk, alice_pbk = eg.keyGen()
bob_pvk, bob_pbk = eg.keyGen()
print("Alice's keys: ", alice_pvk, alice_pbk)
print("Bob's keys: ", bob_pvk, bob_pbk)

Alice's keys:  725306253566939538350339759035304450225393048735 130172683673329459811425123980804042371729310649
Bob's keys:  723980873475542303128283662530441662973433600440 264028465684998091052251434212250271675483981077


In [3]:
# Sharing of the keys

alice_key, alice_enc = eg.KEM(bob_pbk)
print("Alice: ", alice_key)
bob_key = eg.KRev(bob_pvk, alice_enc)
print("Bob: ", bob_key)


Alice:  3162553001967407053133542160209599834411625141
Bob:  3162553001967407053133542160209599834411625141


### C.
Para esta última alínea tinhamos que transformar o nosso *KEM* num *PKE-IND-CCA*

Dado isto constuímos a classe *FO_ElGamal*, que constrói o seu próprio *KEM-ElGamal* que depois utiliza nas funções *encrypt* e *decrypt*.

A função de cifração é definida da seguinte forma:


$E'(x)\;\equiv\;\vartheta\,r \gets h\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\, (e,k) \gets f(y\|r)\,\centerdot\,\vartheta\,c\gets k\oplus r\,\centerdot\,(y, e, c)$

A função de decifração será da seguinte forma:

$D'(y,e,c) \;\equiv\;\vartheta\,k \gets \mathsf{KREv}(e)\,\centerdot\,\vartheta\,r \gets c \oplus k\,\centerdot\,\mathsf{if}\;\;(e,k)\neq f(y\|r) \;\;\mathsf{then}\;\;\bot\;\;\mathsf{else}\;\;y \oplus g(r)$


In [4]:
import hashlib
import os

In [5]:
def xor(b, a):
    return bytes([a^^b for a,b in zip(a,b)])

In [6]:
class FO_ElGamal:
    def __init__(self, size):
        self.kem = ElGamal(size)
    
    def encrypt(self,m, key):
        x = m.encode()
        
        r = ZZ.random_element(2, (self.kem.q)-1)
        r = str(r).encode()
        # Cipher r
        gr = hashlib.sha256(r).digest()
        # XOR
        y = xor(x,gr)
        # Concat r and y
        yr = y + r
        yr = int.from_bytes(yr, byteorder='big')
        # KEM
        k, e = self.kem.KEM(key, yr)
        k_ = int(k).to_bytes(len(r), byteorder='big')
        # XOR k and r
        c = xor(k_,r)
        
        return y,e,c
    
    def decrypt(self, y, e, c, pvk, pbk):
        # KRev
        k = self.kem.KRev(pvk, e)
        k_ = int(k).to_bytes(len(c), byteorder='big')
        
        # XOR k and c
        r = xor(k_,c)
        
        # Check if the decryption can be done
        yr = y + r
        yr = int.from_bytes(yr, byteorder='big')
        
        if (k,e) != (self.kem.KEM(pbk, yr)):
            return "Decryption failed"
        
        # Decryption
        g = hashlib.sha256(r).digest()
        pt = xor(g,y)
        
        return pt.decode()
   
        
        
        
        
    

In [7]:
fo = FO_ElGamal(1024)

# Gen Keys
apvk, apbk = fo.kem.keyGen()
bpvk, bpbk = fo.kem.keyGen()

In [8]:
y,e,c = fo.encrypt("TP2 Fujisaki-Okamoto", bpbk)
pt = fo.decrypt(y,e,c, bpvk, bpbk)
pt

'TP2 Fujisaki-Okamoto'